In [33]:
import pandas as pd

In [34]:
import json

def get_content_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = json.load(file)
    return content

In [35]:
toc = get_content_from_json('..\\..\\database_export\\table_of_contents.json')

def get_something(title, something, toc):
    if type(toc) == list:
        for item in toc:
            g = get_something_for_me(title,something, item)
            if g is not None:
                return g
            
def get_something_for_me(title,something, toc):
    if 'title' in toc and toc['title'] == title and something in toc:
        return toc[something]
    if 'contents' in toc:
        for item in toc['contents']:
            g= get_something_for_me(title,something, item)
            if g is not None:
                return g
        
print(get_something('Genesis','heShortDesc', toc))

בריאת העולם, תחילתה של האנושות וסיפורי האבות והאמהות.


In [36]:

import os

json_folder = "..\\..\\database_export\\json"
schemas_folder = "..\\..\\database_export\\schemas"

data = []

with open("blacklist.txt", 'r', encoding='utf-8') as file:
    blacklist = file.read().splitlines()
    for root, _,files in os.walk(json_folder):
        for file in files:
            file_path = os.path.join(root, file)
            if  file_path.endswith('Hebrew\\merged.json') and file_path.split('\\')[-3] not in blacklist:
                title = file_path.split('\\')[-3].replace(' ', '_')
                schema_file_name = os.path.join(schemas_folder, title + '.json')
                schema = get_content_from_json(schema_file_name)
                heShortDesc = get_something(schema['title'], 'heShortDesc', toc)
                
                order = get_something(schema['title'], 'order', toc)
                author = schema['authors'][0]['he'] if 'authors' in schema and len(schema['authors']) > 0 and 'he' in schema['authors'][0] else ''
                heTitle = schema['heTitle'] if 'heTitle' in schema else ''
                pubPlace = schema['pubPlaceString']['he'] if 'pubPlaceString' in schema else ''
                pubData = schema['pubDateString']['he'] if 'pubDateString' in schema else ''
                compPlace = schema['compPlaceString']['he'] if 'compPlaceString' in schema else ''
                compDate = schema['compDateString']['he'] if 'compDateString' in schema else ''
                heDesc = schema['heDesc'] if 'heDesc' in schema else ''
                data.append({"title": heTitle,"author": author, "pubDate": pubData,'pubPlace': pubPlace, 'compPlace': compPlace, 'compDate': compDate, 'heShortDesc': heShortDesc, 'order': order, 'heDesc': heDesc})



In [37]:
def recursive_register_categories(category):
    if type(category) == list:
        for item in category:
            recursive_register_categories(item)
    if 'contents' in category:
        for item in category['contents']:
            recursive_register_categories(item)
    if 'heCategory' in category:
        heCategory = category['heCategory'] 
        heShortDesc = category['heShortDesc'] if 'heShortDesc' in category else ''
        heDesc = category['heDesc'] if 'heDesc' in category else ''
        order = category['order'] if 'order' in category else ''
        data.append({"title": heCategory,'heDesc': heDesc ,"heShortDesc": heShortDesc, "order": order})

recursive_register_categories(toc)

In [38]:
df = pd.DataFrame(data)
df
                   

,title,author,pubDate,pubPlace,compPlace,compDate,heShortDesc,order,heDesc
0,"חיי מוהר""ן",נתן שטרנהרץ,(1874 לספירה),למברג,,(1810 – 1830 לספירה בקירוב),ביוגרפיה של רבי נחמן שכתב תלמידו רבי נתן.,None,"תורות וקורות חייו של ר' נחמן מברסלב, נכתב ע""י ..."
1,ליקוטי עצות,נחמן מברסלב‎,(1816 לספירה),דובנו,,(1790 – 1810 לספירה בקירוב),אוסף דרשותיו של רבי נחמן העוסקות במידות טובות ...,None,
2,ליקוטי הלכות,נתן שטרנהרץ,(1843 – 1861 לספירה),,,(1814 – 1844 לספירה בקירוב),חיבורו רחב היריעה של רבי נתן המבאר את הלכות הש...,None,
3,"ליקוטי מוהר""ן",נחמן מברסלב‎,(1808 לספירה),אוסטראה,ברסלב,(1802 – 1808 לספירה בקירוב),"יצירת מופת של רבי נחמן, הכוללת פרשנות לאגדות ה...",None,
4,ליקוטי תפילות,נתן שטרנהרץ,(1821 לספירה),ורוצלב,ורוצלב,(1810 – 1820 לספירה בקירוב),״ליקוטי תפילות״ הוא אוסף מהמאה ה-19 הכולל 210 ...,None,״ליקוטי תפילות״ הוא אוסף 210 תפילות שחיבר רבי ...
...,...,...,...,...,...,...,...,...,...
7031,מילונים,NaN,NaN,NaN,NaN,NaN,,5,
7032,דקדוק,NaN,NaN,NaN,NaN,NaN,,10,
7033,שם הגדולים,NaN,NaN,NaN,NaN,NaN,,,
7034,אנציקלופדיות,NaN,NaN,NaN,NaN,NaN,,,


In [39]:
df['heShortDesc'].unique()

array(['ביוגרפיה של רבי נחמן שכתב תלמידו רבי נתן.',
       'אוסף דרשותיו של רבי נחמן העוסקות במידות טובות ומידות רעות, מסודרות על פי סדר אלפביתי.',
       'חיבורו רחב היריעה של רבי נתן המבאר את הלכות השולחן ערוך לאור תורת חסידות ברסלב.',
       'יצירת מופת של רבי נחמן, הכוללת פרשנות לאגדות התלמוד ולחלקים מן הזוהר לצד דרשות נוספות.',
       '״ליקוטי תפילות״ הוא אוסף מהמאה ה-19 הכולל 210 תפילות אישיות וייחודיות שחיבר רבי נתן מנמירוב, תלמידו של רבי נחמן מברסלב, באוקראינה.',
       'אוסף דרשותיו של רבי נחמן על מידות טובות ומידות רעות, ערוך על פי סדר האלפבית.',
       'ביוגרפיה של רבי נחמן המתמקדת במסעו לארץ ישראל, שכתב תלמידו רבי נתן.',
       'לקט שיחות קצרות ותורות של רבי נחמן שהתפרסמו לראשונה כתוספת לחיבור סיפורי מעשיות.',
       'אוסף מעשיות שסיפר רבי נחמן, פורסם לראשונה בחיבור דו-שפתי (בעברית וביידיש).',
       'פירושים על תרי"ג מצוות ועל תפילות שכתב הצמח צדק, האדמו"ר השלישי של חסידות חב"ד.',
       '״ליקוטי תורה״ הוא ספר מרכזי בחסידות חב״ד שנכתב במאה ה-18 על ידי מייסדה - רבי שניאור ז

In [40]:
df.to_excel('metadata.xlsx',index=False)

In [2]:
import pandas as pd
df = pd.read_excel('edited_metadata.xlsx')

In [3]:
df['תיאור_חדש'].unique()

array([nan, 'ברייתא דמסכת אבות',
       'פירוש לתורה ולנביאים מן המאה ה-15; כל קטע חדש בו נפתח ברשימת שאלות על הפסוקים.',
       'ג', 'אגדות חז"ל על התורה',
       'חיבור שכתב הרב שמשון רפאל הירש במאה ה-19, המגן על היהדות ומנוסח כ-19 מכתבים בין רב לצעיר משכיל.',
       'חיבור מן המאה ה-12 שבו מגן רבי אברהם אבן-עזרא על עמדת היהדות , לפיה השבת מתחילה ביום שישי בערב.',
       ' (987 לספירה)',
       'פירוש של הבן איש חי על פסוקים נבחרים מהתורה ומההפטרות בשילוב קבלה, מן המאה ה-18.',
       'חיבור מהמאה ה-15 שכתב רבי חסדאי קרשקש, המגן על עיקרי האמונה ומותח ביקורת על הניסיונות ליישב בין תורת אריסטו ובין יסודות היהודית.',
       'פירוש לתורה מן המאה ה-18 שמקורו במרוקו, ונתפס כיצירה מהותית בעיני מייסדי תנועת החסידות באירופה.',
       'דיונים על תורת הגמול (צדיק ורע לו).', 'קינה על חורבן ירושלים.',
       'פירוש מן המאה ה-16 המדגיש מסרים בתחום טוהר המידות והמוסר הטמונים בכל פסוק.',
       'מגילת אסתר',
       'חיבור המגן על המדרשים ועל האגדות שבתלמוד מפני ביקורת יהודים ושאינם יהודים.',
       "

In [4]:
df['heShortDesc'] = df['תיאור_חדש'].fillna(df['heShortDesc'])

In [5]:
df.to_excel('new_metadata.xlsx',index=False)

In [2]:
import pandas as pd
df = pd.read_excel('metadata.xlsx')
df.to_json('metadata.json', orient='records')